## Function calling using OpenSource LLM model (llama-3) using groq and Langchain
1. Load LLMs
2. Define Tools ( in langchain functions are refered as tools )
3. Link Tools with LLMs
4. Use LLM with Tools
5. Create independent Decision Taking Agent

**Installation**
- Install the necessary packages. I have used Poetry to create virtual env and manage dependencies.

In [1]:
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv())
groq_api_key = os.environ.get("GROQ_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI
llama3 = ChatOpenAI(model="llama3-8b-8192",
                    base_url="https://api.groq.com/openai/v1",
                    api_key = groq_api_key,
                    temperature=0.1)

In [7]:
# test the llm with a invoke
ai_msg = llama3.invoke("Explain about ResultPattern in .Net in four or 5 lines")
ai_msg = ai_msg.content

In [8]:
ai_msg

'In .NET, `ResultPattern` is a feature introduced in C# 9.0 that allows you to handle exceptions and errors in a more concise and expressive way. It provides a way to specify a pattern to match against the result of an expression, and execute a block of code if the pattern matches. This can be used to simplify error handling and make code more readable.'

## 2.Define Tools ##

In [19]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Add two integers and return the result.
       Args:
         a: The first integer to add.
         b: The second integer to add.
       Returns:
         The sum of the two integers, or None if either input is invalid.
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers and return the result.
       Args:
         a: The first integer to multiply.
         b: The second integer to multiply.
       Returns:
         The product of the two integers, or None if either input is invalid.
    """
    return a * b
tools = [add, multiply]

### 3. Link tools to LLM 

In [20]:
llama3_with_tools = llama3.bind_tools(tools)
result = llama3_with_tools.invoke("What is (12+12) x 10 ?")
result.tool_calls

[{'name': 'add',
  'args': {'a': 12, 'b': 12},
  'id': 'call_e51v',
  'type': 'tool_call'},
 {'name': 'multiply',
  'args': {'a': 24, 'b': 10},
  'id': 'call_36vr',
  'type': 'tool_call'}]

In [21]:
llama3_with_tools.to_json()['kwargs']['kwargs']['tools']

[{'type': 'function',
  'function': {'name': 'add',
   'description': 'Add two integers and return the result.\n       Args:\n         a: The first integer to add.\n         b: The second integer to add.\n       Returns:\n         The sum of the two integers, or None if either input is invalid.',
   'parameters': {'properties': {'a': {'type': 'integer'},
     'b': {'type': 'integer'}},
    'required': ['a', 'b'],
    'type': 'object'}}},
 {'type': 'function',
  'function': {'name': 'multiply',
   'description': 'Multiply two integers and return the result.\n       Args:\n         a: The first integer to multiply.\n         b: The second integer to multiply.\n       Returns:\n         The product of the two integers, or None if either input is invalid.',
   'parameters': {'properties': {'a': {'type': 'integer'},
     'b': {'type': 'integer'}},
    'required': ['a', 'b'],
    'type': 'object'}}}]

In [22]:
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_e51v', 'function': {'arguments': '{"a":12,"b":12}', 'name': 'add'}, 'type': 'function'}, {'id': 'call_36vr', 'function': {'arguments': '{"a":24,"b":10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 125, 'prompt_tokens': 2305, 'total_tokens': 2430, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': -0.551996454, 'prompt_time': 0.362100982, 'completion_time': 0.104166667, 'total_time': 0.466267649}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c43c3e5f-8f7b-4fea-8bee-03654b6d2fdc-0', tool_calls=[{'name': 'add', 'args': {'a': 12, 'b': 12}, 'id': 'call_e51v', 'type': 'tool_call'}, {'name': 'multiply', 'args': {'a': 24, 'b': 10}, 'id': 'call_36vr', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2305, 'output_tokens': 125, 'total_tok

### 4. Use LLM with Tools

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage
query = "What is (12*10)+20?"
messages = [HumanMessage(query)]
ai_msg = llama3_with_tools.invoke(query)
messages.append(ai_msg)

# manually we are defining and calling the tools
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add":add,"multiply":multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(
        tool_call_id=tool_call["id"],
        content=str(tool_output)
    ))
messages

[HumanMessage(content='What is (12*10)+20?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_z1g3', 'function': {'arguments': '{"a":12,"b":10}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_fmxs', 'function': {'arguments': '{"a":120,"b":20}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 125, 'prompt_tokens': 2301, 'total_tokens': 2426, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': -0.474444328, 'prompt_time': 0.317781362, 'completion_time': 0.104166667, 'total_time': 0.421948029}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cefed851-3838-4696-83d0-aacb5b603576-0', tool_calls=[{'name': 'multiply', 'args': {'a': 12, 'b': 10}, 'id': 'call_z1g3', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 120, 'b': 20}, 'id': 'call_fmxs', 

In [28]:
final_response = llama3.invoke(messages)
print(final_response.content)

So, the calculation is:

(12*10) = 120
+ 20 = 140

So, the final answer is: 140


### 5. Create Independent Decision Taking Agent
- Use Agent to automate the decision making to select the required tools

In [30]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. You may not need to use tools for every query- the user may just want to chat!"),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
llama3 = ChatOpenAI(model="llama3-70b-8192", 
                    base_url="https://api.groq.com/openai/v1",
                    api_key = groq_api_key,
                    temperature=0.0)
agent = create_tool_calling_agent(llama3, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [31]:
from langchain_core.messages import HumanMessage
agent_executor.invoke({"messages":[HumanMessage(content="Add 10 and 15 and then multiply by 4")]})




> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 10, 'b': 15}`


25
Invoking: `multiply` with `{'a': 25, 'b': 4}`


100The result is 100.

> Finished chain.


{'messages': [HumanMessage(content='Add 10 and 15 and then multiply by 4', additional_kwargs={}, response_metadata={})],
 'output': 'The result is 100.'}

In [33]:
agent_executor.invoke({"messages":[HumanMessage(content="What is 3*12? Also what is 34+89?")]})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 3, 'b': 12}`


36
Invoking: `add` with `{'a': 34, 'b': 89}`


123The answer to 3*12 is 36 and the answer to 34+89 is 123.

> Finished chain.


{'messages': [HumanMessage(content='What is 3*12? Also what is 34+89?', additional_kwargs={}, response_metadata={})],
 'output': 'The answer to 3*12 is 36 and the answer to 34+89 is 123.'}

In [34]:
agent_executor.invoke({"messages":[HumanMessage(content="Add 121 to 6 then multiply by 2")]})




> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 121, 'b': 6}`


127
Invoking: `multiply` with `{'a': 127, 'b': 2}`


254The final answer is 254.

> Finished chain.


{'messages': [HumanMessage(content='Add 121 to 6 then multiply by 2', additional_kwargs={}, response_metadata={})],
 'output': 'The final answer is 254.'}